In [3]:
import telebot
from telebot import types
import pandas as pd

data = pd.ExcelFile('Расписание экзаменов.xlsx')
class Zapros:
    def __init__(self, nomer_kursa, nazvanie_predmeta, gruppa):
        self.nomer_kursa = nomer_kursa
        self.nazvanie_predmeta = nazvanie_predmeta
        self.gruppa = gruppa

def obrabotka_zaprosa(zapros):
    nuzhniy_list = data.parse(sheet_name=zapros.nomer_kursa)
    nuzhnaya_stroka = nuzhniy_list[(nuzhniy_list["Предмет"] == zapros.nazvanie_predmeta) & 
                                   (nuzhniy_list["Группа"].str.contains(zapros.gruppa))]
    if nuzhnaya_stroka.empty:
        return("По выбранному предмету у Вас нет экзамена. Поздравляем 🎉")
    data_examena = nuzhnaya_stroka.iloc[0]['Дата']
    vremya_examena = nuzhnaya_stroka.iloc[0]['Время']
    format_examena = nuzhnaya_stroka.iloc[0]['Формат']
    auditorii_examena = nuzhnaya_stroka.iloc[0]['Аудитория']
    return(f"Информация об экзамене:\nДата экзамена: {data_examena}\nВремя экзамена: {vremya_examena}\nФормат экзамена: {format_examena}\nАудитории экзамена: {auditorii_examena}")

bot = telebot.TeleBot("6617197222:AAFhnmZkyAS1QphsdMeIxEBU7nCZSb8Fcc4")


@bot.message_handler(commands=["start"])
def beginning(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    button1 = types.KeyboardButton("1 курс")
    button2 = types.KeyboardButton("2 курс")
    button3 = types.KeyboardButton("3 курс")
    button4 = types.KeyboardButton("4 курс")
    markup.row(button1, button2, button3, button4)
    bot.send_message(
        message.chat.id,
        "Выберите курс:",
        reply_markup=markup,
    )
    bot.register_next_step_handler(message, after_first_click)


def after_first_click(message):
    curs = message.text
    first_year_courses = [
        "Математический анализ",
        "Геометрия",
        "Алгебра",
        "Педагогическая мастерская",
        "Тривиум",
        "Введение в дискретную математику и топологию",
    ]
    second_year_courses = [
        "Дифференциальные уравнения",
        "Алгебра",
        "Математический анализ 2",
        "Гладкие многообразия",
        "Мера и интеграл",
        "Педагогическая мастерская",
        "Основы программирования на Python",
    ]
    third_year_courses = [
        "Теория вероятностей",
        "Педагогическая мастерская",
        "Алгебра",
        "Экономика",
        "Основы программирования на Python",
    ]
    forth_year_courses = ["Философия", "Руководство проектной деятельностью учащихся"]
    cources = [
        first_year_courses,
        second_year_courses,
        third_year_courses,
        forth_year_courses,
    ]
    course_number = int(message.text[0])
    markup = types.ReplyKeyboardMarkup(resize_keyboard=False)
    markup.row(*cources[course_number - 1])
    bot.send_message(
        message.chat.id,
        "Выберите предмет:",
        reply_markup=markup,
    )
    bot.register_next_step_handler(message, after_second_click, course_number, curs)


def after_second_click(message, course_number, curs):
    nazvanie_predmeta = message.text
    first_year_groups = ["БМТ231", "БМТ232", "БМТ233", "БМТ234", "БЦМТ231", "БЦМТ232"]
    second_year_groups = ["БМТ221", "БМТ222", "БМТ223", "БМТ224", "БЦМТ221", "БЦМТ222"]
    third_year_groups = ["БМТ211", "БМТ212", "БМТ213", "БЦМТ211"]
    forth_year_groups = ["БМТ201", "БМТ202", "БМТ203", "БЦМТ201"]
    groups = [
        first_year_groups,
        second_year_groups,
        third_year_groups,
        forth_year_groups,
    ]
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    markup.row(*groups[course_number - 1])
    bot.send_message(
        message.chat.id,
        "Выберите группу:",
        reply_markup=markup,
    )
    bot.register_next_step_handler(message, after_third_click, curs, nazvanie_predmeta)


def after_third_click(message, curs, nazvanie_predmeta):
    group = message.text
    zapros = Zapros(curs, nazvanie_predmeta, group)
    bot.send_message(
        message.chat.id,
        obrabotka_zaprosa(zapros)
    )
    bot.send_message(message.chat.id, 'Чтобы продолжить, отправьте любое сообщение')
    bot.register_next_step_handler(message, beginning)


bot.polling(none_stop=True)
